In [1]:
from email.header import Header
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from email.mime.text import MIMEText

In [2]:
def SendEmail(emailcontent):#发送邮件的函数
    asender ='1261520571@qq.com'#发件邮箱
    areceiver='15211097884@163.com' #收件邮箱
    asmtpserver = 'smtp.qq.com' #'smtp.qq.com'#发件邮箱smtp地址
    ausername = '1261520571@qq.com'#'#发件邮箱用户名
    apassword = 'ehevkdghepqxhfjd' #'umgqwturdjycbagg'#发件邮箱密码
    
    msgroot = MIMEMultipart('related')
    msgroot['Subject'] = Header('Trading Signal')#邮件标题
    msgroot['to'] = areceiver
    msgroot['from']	= asender 
    
    thebody=MIMEText('condition met '+ emailcontent+',traded','plain', 'UTF-8') #邮件正文
    thebody["Accept-Language"]="zh-CN"
    thebody["Accept-Charset"]="UTF-8" #"UTF-8"
    msgroot.attach(thebody)
    
    asmtp = smtplib.SMTP()
    asmtp.connect(asmtpserver, 25)
    asmtp.login(ausername, apassword)
    asmtp.sendmail(asender, areceiver.split(','), msgroot.as_string())
    asmtp.quit()
    print("mail sending done.")

In [3]:
emailcontent = "买入 上海机场"
SendEmail(emailcontent)

mail sending done.


### 以下为在joinquant里实现的代码

In [ ]:
# 导入函数库
from jqdata import *
from email.header import Header
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from email.mime.text import MIMEText
import numpy as np

def SendEmail(emailcontent):#发送邮件的函数
    asender ='1261520571@qq.com'#发件邮箱
    areceiver='15211097884@163.com' #收件邮箱
    asmtpserver = 'smtp.qq.com' #'smtp.qq.com'#发件邮箱smtp地址
    ausername = '1261520571@qq.com'#'#发件邮箱用户名
    apassword = 'ehevkdghepqxhfjd' #'umgqwturdjycbagg'#发件邮箱密码
    
    msgroot = MIMEMultipart('related')
    msgroot['Subject'] = Header('交易提醒') #邮件标题
    msgroot['to'] = areceiver
    msgroot['from']	= asender 
    
    thebody=MIMEText(emailcontent,'plain', 'UTF-8') #邮件正文
    thebody["Accept-Language"]="zh-CN"
    thebody["Accept-Charset"]="UTF-8" #"UTF-8"
    msgroot.attach(thebody)
    
    asmtp = smtplib.SMTP()
    asmtp.connect(asmtpserver, 25)
    asmtp.login(ausername, apassword)
    asmtp.sendmail(asender, areceiver.split(','), msgroot.as_string())
    asmtp.quit()
    print("mail sending done.")

# 初始化函数，设定基准等等
def initialize(context):
    # 设定沪深300作为基准
    set_benchmark('000300.XSHG')
    # 开启动态复权模式(真实价格)
    set_option('use_real_price', True)
    # 输出内容到日志 log.info()
    log.info('初始函数开始运行且全局只运行一次')

    # 使用数组来表示当前的状态，1表时已入场，0表时未入场
    g.stock_state = np.array([0,0,0,0,0,1,0])
    g.fund_state = np.array([1,1,1])
    # 跟踪的股票：上海机场、中国国旅、五粮液、恒瑞医药、通策医疗、中顺洁柔、昭衍新药（g.为全局变量）
    g.stock_security = ['600009.XSHG', '601888.XSHG', '000858.XSHE', '600276.XSHG', '600763.XSHG', '002511.XSHE', '603127.XSHG']
    # 要操作的基金：半导体50，通信ETF，医疗ETF
    g.fund_security = ['512760.XSHG', '515880.XSHG', '512170.XSHG']

    # 运行函数（reference_security为运行时间的参考标的；传入的标的只做种类区分，因此传入'000300.XSHG'或'510300.XSHG'是一样的）
    # 收盘后运行
    run_daily(after_market_close, time='after_close', reference_security='000300.XSHG')

## 收盘后运行函数
def after_market_close(context):
    log.info(str('函数运行时间(after_market_close):'+str(context.current_dt.time())))
    #得到当天所有成交记录
    trades = get_trades()
    stock_list = g.stock_security
    
    for i in range(len(stock_list)):
        close_data = get_bars(stock_list[i], count=22, unit='1d', fields=['close'])
        # 取得过去20天的平均价格
        MA20_current = close_data['close'][2:].mean()
        MA20_last = close_data['close'][1:21].mean()
        MA20_llast = close_data['close'][:20].mean()
        
        if MA20_current > MA20_last and MA20_last < MA20_llast :
            if g.stock_state[i] == 0:
                g.stock_state[i] = 1
                emailcontent = "买入 {}, 当前价格: {}"\
                    .format(get_security_info(stock_list[i]).display_name,close_data['close'][-1])
                # print(emailcontent)
                SendEmail(emailcontent)
            else:
                emailcontent = "更新 {} 的止盈点, 当前价格: {}"\
                    .format(get_security_info(stock_list[i]).display_name,close_data['close'][-1])
                # print(emailcontent)
                SendEmail(emailcontent)
        
    fund_list = g.fund_security
    for i in range(len(fund_list)):
        close_data = get_bars(fund_list[i], count=22, unit='1d', fields=['close'])
        # 取得过去20天的平均价格
        MA20_current = close_data['close'][2:].mean()
        MA20_last = close_data['close'][1:21].mean()
        MA20_llast = close_data['close'][:20].mean()
        
        if MA20_current > MA20_last and MA20_last < MA20_llast and g.fund_state[i] == 0:
            g.fund_state[i] = 1
            emailcontent = "买入 {}, 当前价格: {}"\
                    .format(get_security_info(fund_list[i]).display_name,close_data['close'][-1])
            # print(emailcontent)
            SendEmail(emailcontent)
        elif close_data['close'][-1] < MA20_current and g.fund_state[i] == 1:
            g.fund_state[i] = 0
            emailcontent = "卖出 {}, 当前价格: {}"\
                    .format(get_security_info(fund_list[i]).display_name,close_data['close'][-1])
            # print(emailcontent)
            SendEmail(emailcontent)
    
    for _trade in trades.values():
        log.info('成交记录：'+str(_trade))
    log.info('一天结束')
    log.info('##############################################################')